Instalações de bibliotecas necessárias:

OBS: A numpy possui um erro na biblioteca boruta_py, com isso é necessário abrir o arquivo borutapy e retirar todos os np.int, np.float, np.bool e converter para int, float e bool, sucessivamente.

In [1]:
'''!pip install pandas
!pip install sklearn
!pip install boruta
!pip install pgmpy
!pip install networkx
!pip install matplotlib'''

'!pip install pandas\n!pip install sklearn\n!pip install boruta\n!pip install pgmpy\n!pip install networkx\n!pip install matplotlib'

In [2]:
'''!pip install --upgrade --force-reinstall pgmpy'''


'!pip install --upgrade --force-reinstall pgmpy'

In [13]:
#Método para realizar o Boruta, e ao final, montar um DataFrame com a ordem selecionada pelo boruta

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.preprocessing import LabelEncoder

def boruta_feature_order(data_path, target_column):
    # Carregar os dados do CSV
    D = pd.read_csv(data_path)
    variables = D.columns
    print(f'Lista de variáveis: {variables}')

    # Print para verificar os dados antes do processamento
    print("Dados antes do processamento:")
    print(D)
    tipo_antes = (type(D[target_column][0]))
    print(f'Tipo dos dados antes do processamento: {tipo_antes}')
    

    # Mapear os valores nominais para números inteiros únicos
    D_encoded = D.apply(LabelEncoder().fit_transform)
    # Print para verificar os dados após o processamento
    print("Dados após o processamento:")
    print(D_encoded)
    tipo_depois = (type(D_encoded[target_column][0]))
    print(f'Tipo dos dados após o processamento (inteiros): {tipo_depois}')

    # Separar os dados em características (X) e alvo (y)
    X = D_encoded.drop(columns=[target_column])
    y = D_encoded[target_column]

    # Inicializar um classificador Random Forest
    rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

    # Inicializar o Boruta
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2)

    # Ajustar o Boruta aos dados
    boruta_selector.fit(X.values, y.values)
    print("Boruta inicializado com sucesso")

    # Obter as características selecionadas
    print("Variáveis na ordem inicial")
    print(variables)
    
    selected_features = X.columns[boruta_selector.support_]
    print("Features selecionadas")
    print(selected_features)
    # Obter as características não selecionadas
    unselected_features = X.columns[~boruta_selector.support_]
    print("Features não selecionadas")
    print(unselected_features)

    # Combinar características selecionadas e não selecionadas, ordenando-as pela ordem de ranking do Boruta
    all_features = [target_column] + list(selected_features) + list(unselected_features)
    print("Todas as features, incluindo a target: ")
    print(all_features)

    # Reorganizar o DataFrame original de acordo com a ordem das características selecionadas pelo Boruta
    df_reordered = D[all_features]
    
    # Salvar o DataFrame reordenado no formato CSV
    df_reordered.to_csv("data_reordered.csv", index=False)

    print("DataFrame reordenado salvo como data_reordered.csv")
    print("DataFrame criado na ordem das features selecionadas pelo Boruta")
    print(df_reordered)
    
    return all_features

# Caminho para o arquivo CSV e o nome da coluna alvo
data_path = "hailfinder.csv"
target_column = "AMCINInScen"

# Obter todas as características, incluindo a coluna alvo
all_features= boruta_feature_order(data_path, target_column)
print("Features em ordem")
print(all_features)

Lista de variáveis: Index(['AMCINInScen', 'AMDewptCalPl', 'AMInstabMt', 'AMInsWliScen',
       'AreaMeso_ALS', 'AreaMoDryAir', 'Boundaries', 'CapChange', 'CapInScen',
       'CldShadeConv', 'CldShadeOth', 'CombClouds', 'CombMoisture',
       'CombVerMo', 'CompPlFcst', 'CurPropConv', 'Date', 'Dewpoints',
       'InsChange', 'InsInMt', 'InsSclInScen', 'IRCloudCover', 'LatestCIN',
       'LIfr12ZDENSd', 'LLIW', 'LoLevMoistAd', 'LowLLapse', 'MeanRH',
       'MidLLapse', 'MorningBound', 'MorningCIN', 'MountainFcst',
       'MvmtFeatures', 'N0_7muVerMo', 'N34StarFcst', 'OutflowFrMt',
       'PlainsFcst', 'QGVertMotion', 'R5Fcst', 'RaoContMoist', 'RHRatio',
       'SatContMoist', 'Scenario', 'ScenRel3_4', 'ScenRelAMCIN',
       'ScenRelAMIns', 'ScnRelPlFcst', 'SfcWndShfDis', 'SubjVertMo',
       'SynForcng', 'TempDis', 'VISCloudCov', 'WindAloft', 'WindFieldMt',
       'WindFieldPln', 'WndHodograph'],
      dtype='object')
Dados antes do processamento:
      AMCINInScen AMDewptCalPl AMInstabMt

In [16]:
import pandas as pd
from pgmpy.estimators import K2Score
from sklearn.preprocessing import LabelEncoder
from pgmpy.models import BayesianModel, BayesianNetwork
from pgmpy.estimators import HillClimbSearch, K2Score, BayesianEstimator


def load_data(DATA_CSV):
    D = pd.read_csv('data_reordered.csv')
    #D = D.apply(LabelEncoder().fit_transform)

    V = D.columns
    N = len(D.index)
    V_CARD = {v: len(D[v].unique()) for v in V}


    print(f'ARQUIVO: {D}')
    print(f'VARIÁVEIS: {V}')
    print(f'NÚMERO DE AMOSTRAS: {N}')
    print(f'MAPEAMENTO DAS VARIÁVEIS COM NÚMEROS DE VALORES ÚNICOS: {V_CARD}')
    return D, V, N, V_CARD

#Método K2 mais correto, que retorna as CPDs

def calcular_k2(D):
    """
    Calcula a pontuação K2 para os dados fornecidos.

    Args:
    - D: DataFrame contendo os dados.

    Returns:
    - k2score: Objeto K2Score contendo a pontuação K2 calculada.
    """
    k2score = K2Score(D)
    return k2score

def estimar_modelo(D, scoring_method):
    """
    Estima a estrutura do modelo usando o método K2.

    Args:
    - D: DataFrame contendo os dados.
    - scoring_method: Objeto de pontuação a ser usado para estimar a estrutura.

    Returns:
    - best_model: Modelo BayesianModel estimado.
    """
    start_dag = BayesianNetwork([(V[i], V[i+1]) for i in range(len(V)-1)])
    #Para obter sempre o melhor modelo, só retirar o start_dag
    estimator_k2 = HillClimbSearch(D)
    # Calcular o número máximo de arestas possíveis
    max_possible_edges = len(V) * (len(V) - 1) / 2
    print(f'Número máximo de arestas possíveis:{max_possible_edges}')

    #Definir max_iter com base no número máximo de arestas possíveis
    max_iter = min(max_possible_edges, 1000)  # Limite superior para max_iter, por exemplo, 1000
    print(f'Número máximo de iterações:{max_iter}')

    best_model = estimator_k2.estimate(scoring_method='k2score', tabu_length=50,max_indegree= 4, max_iter=max_iter) #max_indegree é o número máximo de pais
    #verificar o max_iter, pois a cada iter, ele add uma aresta, e não é isso o necessário
    return best_model


def tabular_cpd(best_model, D):
    """
    Estima as CPDs para o modelo usando o estimador bayesiano.

    Args:
    - best_model: Modelo BayesianModel estimado.
    - D: DataFrame contendo os dados.

    Returns:
    - cpds: Lista de CPDs estimadas.
    """
    bayesian_network = BayesianModel(best_model)
    estimator = BayesianEstimator(bayesian_network, D)
    cpds = []
    for node in bayesian_network.nodes():
        cpd = estimator.estimate_cpd(node)
        cpds.append(cpd)
    return cpds, bayesian_network

# Dados já reordenados pelo Boruta, e suas propriedades
D = pd.read_csv('data_reordered.csv')
print(D)

V = D.columns
N = len(D.index)


# Calcula a pontuação K2
k2score = calcular_k2(D)
# Estima a estrutura do modelo com o K2
print(f'VARIÁVEIS: {V}')
print(f'NÚMERO DE AMOSTRAS: {N}')
best_model = estimar_modelo(D, k2score)
print(f'Melhor modelo: {best_model}')

# Valor do Score gerado pelo K2
k2_score = k2score.score(best_model)
print(f'Estrutura da rede: {best_model.edges}')
print(f'Valor do score K2: {k2_score}')


# Estima as CPDs e passa a bayesian_netq
cpds, bayesian_network = tabular_cpd(best_model, D)
print(f'CPDs: {cpds}')
print(f'Bayesian Network: {bayesian_network}')
#Juntar o best_model com as cpds, para passar para o arquivoxml

      AMCINInScen    CapInScen   MorningCIN ScnRelPlFcst AMDewptCalPl  \
0         Average  LessThanAve          NaN            C    Stability   
1         Average  MoreThanAve  PartInhibit            K    Stability   
2         Average  MoreThanAve  PartInhibit            J    Stability   
3     LessThanAve  LessThanAve          NaN            J    Stability   
4     LessThanAve  MoreThanAve  PartInhibit            A      Neutral   
...           ...          ...          ...          ...          ...   
9995  LessThanAve  LessThanAve  PartInhibit            B      Neutral   
9996      Average      Average          NaN            F    Stability   
9997  MoreThanAve  MoreThanAve     Stifling            H  Instability   
9998      Average  LessThanAve     Stifling            B    Stability   
9999  MoreThanAve  MoreThanAve     Stifling            I    Stability   

     AMInstabMt  AMInsWliScen AreaMeso_ALS AreaMoDryAir Boundaries  ...  \
0           NaN  MoreUnstable       WeakUp      

 20%|██        | 205/1000 [01:43<06:41,  1.98it/s]


Melhor modelo: DAG with 56 nodes and 203 edges


Estrutura da rede: [('ScnRelPlFcst', 'ScenRel3_4'), ('AMInstabMt', 'InsInMt'), ('AreaMeso_ALS', 'CombVerMo'), ('CapChange', 'CompPlFcst'), ('CldShadeConv', 'InsInMt'), ('CurPropConv', 'LLIW'), ('CurPropConv', 'SfcWndShfDis'), ('CurPropConv', 'Dewpoints'), ('CurPropConv', 'WindFieldPln'), ('CurPropConv', 'Date'), ('CurPropConv', 'SynForcng'), ('CurPropConv', 'N0_7muVerMo'), ('CurPropConv', 'WindAloft'), ('CurPropConv', 'AreaMoDryAir'), ('CurPropConv', 'LowLLapse'), ('CurPropConv', 'CombMoisture'), ('CurPropConv', 'LoLevMoistAd'), ('CurPropConv', 'SatContMoist'), ('CurPropConv', 'RaoContMoist'), ('CurPropConv', 'MvmtFeatures'), ('CurPropConv', 'AreaMeso_ALS'), ('CurPropConv', 'SubjVertMo'), ('CurPropConv', 'QGVertMotion'), ('CurPropConv', 'ScenRelAMIns'), ('CurPropConv', 'CapInScen'), ('CurPropConv', 'LIfr12ZDENSd'), ('CurPropConv', 'InsChange'), ('CurPropConv', 'AMInsWliScen'), ('CurPropConv', 'MidLLapse'), ('CurPropConv', 'CapChange'), ('CurPropConv', 'InsSclInScen'), ('CurPropConv', '

In [17]:
from pgmpy.readwrite import XMLBIFWriter

# Inicialize o Modelo Bayesian
modelo_completo = BayesianModel(bayesian_network)  # Inicialize com as arestas do bayesian_network
print(f'Modelo: {modelo_completo}')

# Adicionando as CPDs ao Modelo
for cpd in cpds:
    modelo_completo.add_cpds(cpd)

# Verifique a consistência do modelo
assert modelo_completo.check_model()

# Verificar a Estrutura do Modelo
print("Estrutura do Modelo:")
print(modelo_completo.edges())

# Verificar as CPDs e imprimir na tela
'''print("\nCPDs do Modelo:")
for node in modelo_completo.nodes():
    print(f"CPD para o nó {node}:")
    print(modelo_completo.get_cpds(node))'''

# Verificar a Consistência do Modelo
print("\nVerificação de Consistência do Modelo:")
print(modelo_completo.check_model())

# Fazendo o formato XMLBIF
writer = XMLBIFWriter(modelo_completo)

# Especifique o nome do arquivo para salvar o modelo
nome_arquivo = "hailfinder.xml"

# Escreva o modelo no arquivo .xmlbif
writer.write_xmlbif(nome_arquivo)


Modelo: BayesianModel with 56 nodes and 203 edges
Estrutura do Modelo:
[('ScnRelPlFcst', 'ScenRel3_4'), ('AMInstabMt', 'InsInMt'), ('AreaMeso_ALS', 'CombVerMo'), ('CapChange', 'CompPlFcst'), ('CldShadeConv', 'InsInMt'), ('CurPropConv', 'LLIW'), ('CurPropConv', 'SfcWndShfDis'), ('CurPropConv', 'Dewpoints'), ('CurPropConv', 'WindFieldPln'), ('CurPropConv', 'Date'), ('CurPropConv', 'SynForcng'), ('CurPropConv', 'N0_7muVerMo'), ('CurPropConv', 'WindAloft'), ('CurPropConv', 'AreaMoDryAir'), ('CurPropConv', 'LowLLapse'), ('CurPropConv', 'CombMoisture'), ('CurPropConv', 'LoLevMoistAd'), ('CurPropConv', 'SatContMoist'), ('CurPropConv', 'RaoContMoist'), ('CurPropConv', 'MvmtFeatures'), ('CurPropConv', 'AreaMeso_ALS'), ('CurPropConv', 'SubjVertMo'), ('CurPropConv', 'QGVertMotion'), ('CurPropConv', 'ScenRelAMIns'), ('CurPropConv', 'CapInScen'), ('CurPropConv', 'LIfr12ZDENSd'), ('CurPropConv', 'InsChange'), ('CurPropConv', 'AMInsWliScen'), ('CurPropConv', 'MidLLapse'), ('CurPropConv', 'CapChange')

TESTE:

1. Rodar 20x o K2 com ordem aleatória, para cada base de dados, e no fim tirar a média da função G (score), e o tempo;
    1.1 Guardar só a média das 20x;

2. Deixar anotado em uma planilha;

3. Rodar o K2 com o Boruta, para cada variável sendo classe, e a execução com o melhor G, eu guardo, guardando o tempo;

4. Guardar o XML;